In [160]:
from inventory import *
import requests
from bs4 import BeautifulSoup
from PIL import Image
import os.path
from os import path
from random import randrange

In [2]:
# get links for all mezlan shoes
links = []

result = requests.get("https://www.mezlan.com/shoes")

prev = ''
i = 1
while result.status_code == 200:
    result = requests.get("https://www.mezlan.com/shoes?p=" + str(i))
    soup = BeautifulSoup(result.content, "html.parser")
    a = soup.find_all("h2", {"class": "product-name"})
    first = a[0].find("a").text
    if prev == first:
        break
    prev = first
    for j in range(len(a)):
        links.append(a[j].find("a")['href'])
    i += 1

- Boot
    - boot
    - belucci   
- Dress Casual Shoe
    - monk
    - sneaker
    - dress-casual 
    
- Dress Shoe
    - lace up
    - lace-up
    - oxford
    - blucher
    - balmoral
    
- Loafer and Slip-On
    - loafer
    - slip on
    - moccasin
    - mocassin
    - pump
    - porty
    - rollini

In [193]:
def getType(a):
    if any(w in a.lower() for w in ['boot', 'belucci']):
        return 'Boot'
    if any(w in a.lower() for w in ['monk', 'sneaker', 'dress-casual']):
        return 'Dress Casual Shoe'
    if any(w in a.lower() for w in ['loafer', 'slip on', 'moccasin', 'mocassin', 'pump', 'porty', 'rollini']):
        return 'Loafer and Slip On'
    return 'Dress Shoe'

In [326]:
def saveSwatch(color, imglink):
    # finds and downloads swatches
    response = requests.get(imglink)
    try:
        image = Image.open(BytesIO(response.content))
    except:
        return
    x, y = image.size
    xl = x//2 - x//20
    xr = xl + x//10
    yl = 13*y//16 - y//20
    yr = yl + y//10
    image = image.crop((xl, yl, xr, yr))
    pt = "swatches/"+ (color.lower().replace(' ', '-').replace('/', '-'))

#     if path.exists(pt):
    image.save(pt + str(randrange(10)) +".png")
#     else:
#         image.save(pt +".png")

In [331]:
def getProduct(url):
    res = requests.get(url)
    if res.status_code != 200:
        print("URL Error: " + url)
        return None
    soup = BeautifulSoup(res.content, "html.parser")
    
#     PRE-KNOWN INFORMATION
    vendor = "Mezlan"

    
#     title
    title = soup.title.text
    title = title.title()
    
#     handle
    handle = vendor.lower()+title.replace(' ', '').lower().replace("-", "").replace("/", "")
    
#     type
    ptype = soup.find("div", {"class": "std"}).find("ul").find_all("li")[0].text
    type = getType(str(title + " " + ptype))
    
#     seo title
    seotitle = 'Brite Creations | ' + title + " " + type +' by ' + vendor
    
#     body
    body = soup.find("div", {"class": "std"}).find("ul")
    body = str(body)
    body = "<meta charset=\"utf-8\">\n" + body[:-5] + "\n<li>Best quality and price from <a href=\"https://britecreations.com/pages/about\">Brite Creations Atlanta</a>.</li>\n</ul>"
    
#     seo description
    seodescription = 'Shop at Brite Creations Atlanta for the best deal on ' + title + " " + type +' by ' + vendor + '\n' + body

#     sizes, colors, images
    colsiz = soup.find_all("span", {"class": "swatch-label"})[:-1]
    sizes = [str(a.text.replace(' ', '').replace('\n', '').replace('½', '.5')).replace('.0', '') for a in colsiz if a.text.replace('\n', '') != '' and not ('A' in str(a) or 'E' in str(a) or 'I' in str(a) or 'O' in str(a) or 'U' in str(a))]
    try:
        colors = [a.find("img")['alt'].title() for a in colsiz if ('A' in str(a) or 'E' in str(a) or 'I' in str(a) or 'O' in str(a) or 'U' in str(a))]
        colors = [c for c in colors if c != '']
    except:
        colors = [str(a.text.replace(' ', '').replace('\n', '')).title() for a in colsiz if ('A' in str(a) or 'E' in str(a) or 'I' in str(a) or 'O' in str(a) or 'U' in str(a))]
        colors = [c for c in colors if c != '']
    images = soup.find_all("img", {"class": "gallery-image"})
    images = [a['src'] for a in images if '1800x' in a['src']]
    
    finalcolors = []
    finalimages = []
    
    for i in images:
        i = i.replace(colors[0].lower(), 'XXXXX')
        for c in colors:
            img = i.replace('XXXXX', c.lower())
            try:
                img = img[:img.index("/cache/")] + img[img.index("/m/e/"):] 
            except:
                img = img[:img.index("/cache/")] + img[img.index("/b/a/"):] 
            res = requests.get(img)
            if c not in [cl for cl, im in finalcolors]:
                finalcolors.append((c, img))
            if res.status_code == 200:
                if img not in [im for im, at in finalimages]:
                    finalimages.append((img, ''))

    colors = finalcolors
    images = finalimages
    
#     tags
    tags = []
    for c, i in colors:
        tag = 'color:' + c
        if tag not in tags:
            tags.append(tag)
    for s in sizes:
        tag = 'size:' + str(s).replace('.0', '')
        if tag not in tags:
            tags.append(tag)

#     price
    p = float(soup.find("span", {"class": "price"}).text.replace(',', '').replace('$', ''))
    op = p - 20
    
    price = []
    oldprice = []
    for i in range(len(colors) * len(sizes)):
        price.append(p)
        oldprice.append(op)
    
#     inventory and sku
    inventory = []
    sku = []
    
    for i in range(len(colors) * len(sizes)):
        inventory.append((1, 0)) #(online, retail)
    
    for s in sizes:
        for c, i in colors:
            sku.append(handle+"-"+c.lower()+"-"+str(s).replace('.0', ''))
    
#     swatches
    for c, i in colors:
        saveSwatch(c, i)
    
    prod = product(handle=handle, title=title, body=body, vendor=vendor, type=type, sizes=sizes, 
                   colors=colors, images=images, tags=tags, price=price, oldprice=oldprice, seotitle=seotitle, 
                   seodescription=seodescription, sku=sku, inventory=inventory)
    return prod

In [333]:
products = []
for link in links:
    p = getProduct(link)
    products.append(p)
    print(p)
print(len(products))

Tibero
----------------------------------
Handle: 	mezlantibero
Sizes: 		['8', '8.5', '9', '9.5', '10', '10.5', '11', '11.5', '12', '13', '14']
Colors: 	[('Burgundy', 'https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-7276-f-tibero-burgundy-1.1603945711.jpg')]
SKU: 		['mezlantibero-burgundy-8', 'mezlantibero-burgundy-8.5', 'mezlantibero-burgundy-9', 'mezlantibero-burgundy-9.5', 'mezlantibero-burgundy-10', 'mezlantibero-burgundy-10.5', 'mezlantibero-burgundy-11', 'mezlantibero-burgundy-11.5', 'mezlantibero-burgundy-12', 'mezlantibero-burgundy-13', 'mezlantibero-burgundy-14']
Images: 	[('https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-7276-f-tibero-burgundy-1.1603945711.jpg', ''), ('https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-7276-f-tibero-burgundy-ou.1603945711.jpg', ''), ('https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-7276-f-tibero-burgundy-2.1603945711.jpg', ''), ('https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes

Newell
----------------------------------
Handle: 	mezlannewell
Sizes: 		['9', '9.5', '10', '10.5', '11', '11.5', '12', '13', '14', '15']
Colors: 	[('Black', 'https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-9736-newell-black-1.1603945246.jpg')]
SKU: 		['mezlannewell-black-9', 'mezlannewell-black-9.5', 'mezlannewell-black-10', 'mezlannewell-black-10.5', 'mezlannewell-black-11', 'mezlannewell-black-11.5', 'mezlannewell-black-12', 'mezlannewell-black-13', 'mezlannewell-black-14', 'mezlannewell-black-15']
Images: 	[('https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-9736-newell-black-1.1603945246.jpg', ''), ('https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-9736-newell-black-ou.1603945246.jpg', ''), ('https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-9736-newell-black-2.1603945246.jpg', ''), ('https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-9736-newell-black-3.1603945247.jpg', '')]
-----------------------------------

Enterpr

Gehry
----------------------------------
Handle: 	mezlangehry
Sizes: 		['8', '8.5', '9', '9.5', '10', '10.5', '11', '11.5', '12', '13']
Colors: 	[('Black', 'https://www.mezlan.com/media/catalog/product/b/a/bacco-bucci-shoes-8205-15-gehry-black-1.1603944656.jpg'), ('Tan', 'https://www.mezlan.com/media/catalog/product/b/a/bacco-bucci-shoes-8205-15-gehry-tan-1.1603944656.jpg')]
SKU: 		['mezlangehry-black-8', 'mezlangehry-tan-8', 'mezlangehry-black-8.5', 'mezlangehry-tan-8.5', 'mezlangehry-black-9', 'mezlangehry-tan-9', 'mezlangehry-black-9.5', 'mezlangehry-tan-9.5', 'mezlangehry-black-10', 'mezlangehry-tan-10', 'mezlangehry-black-10.5', 'mezlangehry-tan-10.5', 'mezlangehry-black-11', 'mezlangehry-tan-11', 'mezlangehry-black-11.5', 'mezlangehry-tan-11.5', 'mezlangehry-black-12', 'mezlangehry-tan-12', 'mezlangehry-black-13', 'mezlangehry-tan-13']
Images: 	[('https://www.mezlan.com/media/catalog/product/b/a/bacco-bucci-shoes-8205-15-gehry-black-1.1603944656.jpg', ''), ('https://www.mezlan.co

Avignon
----------------------------------
Handle: 	mezlanavignon
Sizes: 		['8', '8.5', '9', '9.5', '10', '10.5', '11', '11.5', '12', '13', '14']
Colors: 	[('Black', 'https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-7310-s-avignon-brandy-1.1603946226.jpg'), ('Brandy', 'https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-7310-s-avignon-brandy-1.1603946226.jpg')]
SKU: 		['mezlanavignon-black-8', 'mezlanavignon-brandy-8', 'mezlanavignon-black-8.5', 'mezlanavignon-brandy-8.5', 'mezlanavignon-black-9', 'mezlanavignon-brandy-9', 'mezlanavignon-black-9.5', 'mezlanavignon-brandy-9.5', 'mezlanavignon-black-10', 'mezlanavignon-brandy-10', 'mezlanavignon-black-10.5', 'mezlanavignon-brandy-10.5', 'mezlanavignon-black-11', 'mezlanavignon-brandy-11', 'mezlanavignon-black-11.5', 'mezlanavignon-brandy-11.5', 'mezlanavignon-black-12', 'mezlanavignon-brandy-12', 'mezlanavignon-black-13', 'mezlanavignon-brandy-13', 'mezlanavignon-black-14', 'mezlanavignon-brandy-14']
Images: 	[('ht

Toronado
----------------------------------
Handle: 	mezlantoronado
Sizes: 		['8', '8.5', '9', '9.5', '10', '10.5', '11', '11.5', '12', '13']
Colors: 	[('Black', 'https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-9467-toronado-cognac-1.1603947399.jpg'), ('Cognac', 'https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-9467-toronado-cognac-1.1603947399.jpg')]
SKU: 		['mezlantoronado-black-8', 'mezlantoronado-cognac-8', 'mezlantoronado-black-8.5', 'mezlantoronado-cognac-8.5', 'mezlantoronado-black-9', 'mezlantoronado-cognac-9', 'mezlantoronado-black-9.5', 'mezlantoronado-cognac-9.5', 'mezlantoronado-black-10', 'mezlantoronado-cognac-10', 'mezlantoronado-black-10.5', 'mezlantoronado-cognac-10.5', 'mezlantoronado-black-11', 'mezlantoronado-cognac-11', 'mezlantoronado-black-11.5', 'mezlantoronado-cognac-11.5', 'mezlantoronado-black-12', 'mezlantoronado-cognac-12', 'mezlantoronado-black-13', 'mezlantoronado-cognac-13']
Images: 	[('https://www.mezlan.com/media/catalog/prod

...

Dallas
----------------------------------
Handle: 	mezlandallas
Sizes: 		['8', '10.5', '11', '11.5', '12', '13']
Colors: 	[('Black', 'https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-14436-f-dallas-black-1.1603946242.jpg')]
SKU: 		['mezlandallas-black-8', 'mezlandallas-black-10.5', 'mezlandallas-black-11', 'mezlandallas-black-11.5', 'mezlandallas-black-12', 'mezlandallas-black-13']
Images: 	[('https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-14436-f-dallas-black-1.1603946242.jpg', ''), ('https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-14436-f-dallas-black-ou.1603946242.jpg', ''), ('https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-14436-f-dallas-black-2.1603946242.jpg', ''), ('https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-14436-f-dallas-black-3.1603946242.jpg', '')]
-----------------------------------

Damon
----------------------------------
Handle: 	mezlandamon
Sizes: 		['8', '8.5', '9', '9.5', '10', '10.5', '11', '1

Pegaso
----------------------------------
Handle: 	mezlanpegaso
Sizes: 		['8', '8.5', '9', '9.5', '10', '10.5', '11', '11.5', '12', '13', '14']
Colors: 	[('Black', 'https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-4316-l-pegaso-honey-1_3.1603945756.jpg'), ('Honey', 'https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-4316-l-pegaso-honey-1_3.1603945756.jpg')]
SKU: 		['mezlanpegaso-black-8', 'mezlanpegaso-honey-8', 'mezlanpegaso-black-8.5', 'mezlanpegaso-honey-8.5', 'mezlanpegaso-black-9', 'mezlanpegaso-honey-9', 'mezlanpegaso-black-9.5', 'mezlanpegaso-honey-9.5', 'mezlanpegaso-black-10', 'mezlanpegaso-honey-10', 'mezlanpegaso-black-10.5', 'mezlanpegaso-honey-10.5', 'mezlanpegaso-black-11', 'mezlanpegaso-honey-11', 'mezlanpegaso-black-11.5', 'mezlanpegaso-honey-11.5', 'mezlanpegaso-black-12', 'mezlanpegaso-honey-12', 'mezlanpegaso-black-13', 'mezlanpegaso-honey-13', 'mezlanpegaso-black-14', 'mezlanpegaso-honey-14']
Images: 	[('https://www.mezlan.com/media/catalog/p

Capizzi
----------------------------------
Handle: 	mezlancapizzi
Sizes: 		['7.5', '8', '8.5', '9', '9.5', '10', '10.5', '11', '11.5', '12', '13']
Colors: 	[('Black', 'https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-6398-capizzi-black-1_9.1603946800.jpg')]
SKU: 		['mezlancapizzi-black-7.5', 'mezlancapizzi-black-8', 'mezlancapizzi-black-8.5', 'mezlancapizzi-black-9', 'mezlancapizzi-black-9.5', 'mezlancapizzi-black-10', 'mezlancapizzi-black-10.5', 'mezlancapizzi-black-11', 'mezlancapizzi-black-11.5', 'mezlancapizzi-black-12', 'mezlancapizzi-black-13']
Images: 	[('https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-6398-capizzi-black-1_9.1603946800.jpg', ''), ('https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-6398-capizzi-black-ou_9.1603946800.jpg', ''), ('https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-6398-capizzi-black-2_9.1603946800.jpg', '')]
-----------------------------------

Phoenix
----------------------------------
Handle: 	mez

Concerto
----------------------------------
Handle: 	mezlanconcerto
Sizes: 		['7.5', '8', '8.5', '9', '9.5', '10', '10.5', '11', '11.5', '12', '13', '14', '15', 'Medium']
Colors: 	[('Black', 'https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-3802-concerto-black-1_10.1603946800.jpg')]
SKU: 		['mezlanconcerto-black-7.5', 'mezlanconcerto-black-8', 'mezlanconcerto-black-8.5', 'mezlanconcerto-black-9', 'mezlanconcerto-black-9.5', 'mezlanconcerto-black-10', 'mezlanconcerto-black-10.5', 'mezlanconcerto-black-11', 'mezlanconcerto-black-11.5', 'mezlanconcerto-black-12', 'mezlanconcerto-black-13', 'mezlanconcerto-black-14', 'mezlanconcerto-black-15', 'mezlanconcerto-black-Medium']
Images: 	[('https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-3802-concerto-black-1_10.1603946800.jpg', ''), ('https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-3802-concerto-black-ou_10.1603946800.jpg', ''), ('https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-3802-concer

Santander
----------------------------------
Handle: 	mezlansantander
Sizes: 		['7', '7.5', '8', '8.5', '9', '9.5', '10', '10.5', '11', '11.5', '12', '13', '14', '16', '17']
Colors: 	[('Black', 'https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-0544-santander-black-1_6.1604119546.jpg'), ('Burgundy', 'https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-0544-santander-burgundy-1_6.1604119546.jpg'), ('Tan', 'https://www.mezlan.com/media/catalog/product/m/e/mezlan-shoes-0544-santander-tan-1_6.1604119546.jpg')]
SKU: 		['mezlansantander-black-7', 'mezlansantander-burgundy-7', 'mezlansantander-tan-7', 'mezlansantander-black-7.5', 'mezlansantander-burgundy-7.5', 'mezlansantander-tan-7.5', 'mezlansantander-black-8', 'mezlansantander-burgundy-8', 'mezlansantander-tan-8', 'mezlansantander-black-8.5', 'mezlansantander-burgundy-8.5', 'mezlansantander-tan-8.5', 'mezlansantander-black-9', 'mezlansantander-burgundy-9', 'mezlansantander-tan-9', 'mezlansantander-black-9.5', 'mezlan

In [335]:
productstocsv(products)
productstoinventory(products)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
Errors: []
